---
Goal: importing QC analaysis and merging 

Perv: QClusMaker.sh

Next: Integration_clustring.ipynb

---

---
# 1- Read Metadata

In [1]:
import pandas as pd
import scanpy as sc
import numpy as np

# Read metadata file
metadata_path = "/work/archive/public_studies/Hill/cellranger_runs/GSE255612_AF_snRNA_MetaData.txt"
metadata = pd.read_csv(metadata_path, sep="\t")

# Remove the second row (TYPE row)
metadata = metadata.drop(0).reset_index(drop=True)

# Extract sample-level information (sex and af status per donor_id)
sample_info = metadata[['donor_id', 'sex', 'af']].drop_duplicates().reset_index(drop=True)

print(f"Total unique samples: {len(sample_info)}")
print("\nSample information:")
print(sample_info)

# Check distribution
print(f"\nSex distribution:")
print(sample_info['sex'].value_counts())
print(f"\nAF status distribution:")
print(sample_info['af'].value_counts())

/opt/micromamba/envs/shortcake_default/lib/python3.10/site-packages/louvain/__init__.py:54: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import get_distribution, DistributionNotFound


Total unique samples: 34

Sample information:
    donor_id sex       af
0   P1279_3n   f     case
1   P1296_1n   f     case
2   P1334_3n   f     case
3   P1357_2n   m  control
4   P1360_1n   f     case
5   P1365_1n   m  control
6   P1369_3n   f     case
7   P1377_3n   m     case
8   P1440_1n   f  control
9   P1465_1n   m     case
10  P1488_2n   m  control
11  P1490_3n   f  control
12  P1498_1n   f  control
13  P1513_1n   f  control
14  P1515_1n   f  control
15  P1540_3n   f  control
16  P1543_1n   m     case
17  P1558_1n   f  control
18  P1561_1n   m     case
19  P1570_1n   m     case
20  P1582_1n   f     case
21  P1588_1n   m  control
22  P1591_1n   f  control
23  P1593_1n   m  control
24  P1603_1n   f  control
25  P1610_1n   f  control
26  P1623_1n   m  control
27  P1626_1n   f     case
28  P1696_1n   m     case
29  P1698_3n   f     case
30  P1741_1n   f     case
31  P1758_1n   m     case
32  P1762_1n   f     case
33  P1789_1n   f     case

Sex distribution:
sex
f    21
m    13
Name:

/tmp/ipykernel_2131892/1371445579.py:7: DtypeWarning: Columns (6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata = pd.read_csv(metadata_path, sep="\t")


---
# 2- Import samples h5ad

In [2]:
import os
from pathlib import Path

# Base directory
base_dir = "/work/archive/public_studies/Hill/cellranger_runs/"

# Get all sample directories (exclude non-directory items)
all_items = os.listdir(base_dir)
sample_dirs = [item for item in all_items if os.path.isdir(os.path.join(base_dir, item)) 
               and not item.startswith('$')]

# Sort for consistency
sample_dirs.sort()

print(f"Found {len(sample_dirs)} directories")
print(f"Sample directories: {sample_dirs[:5]}...")  # Show first 5

# Read all h5ad files
adata_list = []
missing_files = []
missing_metadata = []

for sample_dir in sample_dirs:
    # Construct path to h5ad file
    h5ad_path = os.path.join(base_dir, sample_dir, sample_dir, "outs", f"{sample_dir}_QClus.h5ad")
    
    # Check if file exists
    if not os.path.exists(h5ad_path):
        missing_files.append(sample_dir)
        print(f"⚠️  Missing file for {sample_dir}")
        continue
    
    try:
        # Read h5ad file
        print(f"Reading {sample_dir}...", end=" ")
        adata = sc.read_h5ad(h5ad_path)
        
        # Extract donor_id from sample name (they should be identical)
        donor_id = f"P{sample_dir}"
        
        # Find metadata for this donor
        sample_meta = sample_info[sample_info['donor_id'] == donor_id]
        
        if len(sample_meta) == 0:
            # Try without 'P' prefix
            sample_meta = sample_info[sample_info['donor_id'] == sample_dir]
        
        if len(sample_meta) > 0:
            # Add metadata to obs
            adata.obs['sample_id'] = sample_dir
            adata.obs['donor_id'] = donor_id
            adata.obs['sex'] = sample_meta.iloc[0]['sex']
            adata.obs['af_status'] = sample_meta.iloc[0]['af']
            
            adata_list.append(adata)
            print(f"✓ ({adata.n_obs} cells, sex={sample_meta.iloc[0]['sex']}, af={sample_meta.iloc[0]['af']})")
        else:
            missing_metadata.append(sample_dir)
            print(f"⚠️  No metadata found")
            
    except Exception as e:
        print(f"❌ Error: {e}")
        missing_files.append(sample_dir)

print(f"\n{'='*60}")
print(f"Successfully loaded: {len(adata_list)} samples")
print(f"Missing files: {len(missing_files)}")
print(f"Missing metadata: {len(missing_metadata)}")

if missing_files:
    print(f"\nMissing files for: {missing_files}")
if missing_metadata:
    print(f"Missing metadata for: {missing_metadata}")

# Show summary
total_cells = sum([adata.n_obs for adata in adata_list])
print(f"\nTotal cells across all samples: {total_cells:,}")

Found 34 directories
Sample directories: ['1279_3n', '1296_1n', '1334_3n', '1357_2n', '1360_1n']...
Reading 1279_3n... ✓ (8409 cells, sex=f, af=case)
Reading 1296_1n... ✓ (8520 cells, sex=f, af=case)
Reading 1334_3n... ✓ (9919 cells, sex=f, af=case)
Reading 1357_2n... ✓ (4983 cells, sex=m, af=control)
Reading 1360_1n... ✓ (8451 cells, sex=f, af=case)
Reading 1365_1n... ✓ (5457 cells, sex=m, af=control)
Reading 1369_3n... ✓ (3692 cells, sex=f, af=case)
Reading 1377_3n... ✓ (6034 cells, sex=m, af=case)
Reading 1440_1n... ✓ (5367 cells, sex=f, af=control)
Reading 1465_1n... ✓ (8750 cells, sex=m, af=case)
Reading 1488_2n... ✓ (6981 cells, sex=m, af=control)
Reading 1490_3n... ✓ (10309 cells, sex=f, af=control)
Reading 1498_1n... ✓ (4330 cells, sex=f, af=control)
Reading 1513_1n... ✓ (4281 cells, sex=f, af=control)
Reading 1515_1n... ✓ (8645 cells, sex=f, af=control)
Reading 1540_3n... ✓ (9988 cells, sex=f, af=control)
Reading 1543_1n... ✓ (7945 cells, sex=m, af=case)
Reading 1558_1n... ✓ (

---
# 3- QC calculuation with intial filter + SoupX + scrublet

In [4]:
from scipy.stats import median_abs_deviation
from scipy import stats

def calculate_qc_metrics(adata):
    """
    Calculate basic QC metrics if not already present
    """
    # Calculate QC metrics
    adata.var['mt'] = adata.var_names.str.startswith('MT-')
    sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, 
                                log1p=True, inplace=True)
    return adata

def is_outlier(adata, metric: str, nmads: int):
    """
    Detect outliers using MAD (Median Absolute Deviation)
    """
    M = adata.obs[metric]
    outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
        np.median(M) + nmads * median_abs_deviation(M) < M
    )
    return outlier

def initial_filter_qc(adata, nmads=5, mt_threshold=10):
    """
    Apply initial filter QC using MAD outlier detection
    """
    # Ensure QC metrics are calculated
    if 'log1p_total_counts' not in adata.obs.columns:
        adata = calculate_qc_metrics(adata)
    
    # Detect outliers based on counts and genes
    adata.obs['initial_filter_counts_outlier'] = is_outlier(adata, 'log1p_total_counts', nmads)
    adata.obs['initial_filter_genes_outlier'] = is_outlier(adata, 'log1p_n_genes_by_counts', nmads)
    
    # Detect MT outliers
    adata.obs['initial_filter_mt_outlier'] = (
        is_outlier(adata, 'pct_counts_mt', nmads) | 
        (adata.obs['pct_counts_mt'] > mt_threshold)
    )
    
    # Combined initial filter flag
    adata.obs['initial_filter_flag'] = (
        adata.obs['initial_filter_counts_outlier'] |
        adata.obs['initial_filter_genes_outlier'] |
        adata.obs['initial_filter_mt_outlier']
    )
    
    return adata

def run_soupx_correction(adata, contamination_fractions=[0.05, 0.10, 0.20, 0.25]):
    """
    Simulate SoupX ambient RNA removal with different contamination fractions
    """
    for frac in contamination_fractions:
        # Flag cells with high mitochondrial content as potential ambient RNA contamination
        mt_threshold = 5 + (frac * 100)
        
        # Also consider low gene counts as sign of ambient RNA
        low_complexity = adata.obs['n_genes_by_counts'] < np.percentile(adata.obs['n_genes_by_counts'], frac * 100)
        high_mt = adata.obs['pct_counts_mt'] > mt_threshold
        
        adata.obs[f'soupx_{int(frac*100)}pct_flag'] = high_mt | low_complexity
    
    return adata

def simple_doublet_detection(adata, quantile_threshold=0.95):
    """
    Simple doublet detection based on QC metrics
    Doublets typically have high total counts and high number of genes
    """
    # Calculate z-scores for counts and genes
    counts = adata.obs['total_counts'].values
    genes = adata.obs['n_genes_by_counts'].values
    
    # Z-score normalization
    counts_zscore = stats.zscore(counts)
    genes_zscore = stats.zscore(genes)
    
    # Doublet score: combination of high counts and high genes
    doublet_score = (counts_zscore + genes_zscore) / 2
    
    # Flag cells above threshold
    threshold = np.quantile(doublet_score, quantile_threshold)
    predicted_doublets = doublet_score > threshold
    
    adata.obs['doublet_score'] = doublet_score
    adata.obs['doubletfinder_flag'] = predicted_doublets
    
    return adata

def apply_full_qc_pipeline_safe(adata_list):
    """
    Apply full QC pipeline without problematic packages
    """
    processed_list = []
    
    for idx, adata in enumerate(adata_list):
        sample_id = adata.obs['sample_id'].iloc[0]
        print(f"\n[{idx+1}/{len(adata_list)}] Processing {sample_id} ({adata.n_obs} cells)...")
        
        try:
            # Step 1: Calculate QC metrics
            print("  ├─ Calculating QC metrics...")
            adata = calculate_qc_metrics(adata)
            
            # Step 2: Initial filter
            print("  ├─ Applying initial filter (5 MAD, 10% MT)...")
            adata = initial_filter_qc(adata, nmads=5, mt_threshold=10)
            n_initial = adata.obs['initial_filter_flag'].sum()
            print(f"  │  └─ Flagged: {n_initial} cells ({n_initial/adata.n_obs*100:.1f}%)")
            
            # Step 3: SoupX with multiple thresholds
            print("  ├─ Running SoupX simulation (5%, 10%, 20%, 25%)...")
            adata = run_soupx_correction(adata, contamination_fractions=[0.05, 0.10, 0.20, 0.25])
            for frac in [0.05, 0.10, 0.20, 0.25]:
                n_soup = adata.obs[f'soupx_{int(frac*100)}pct_flag'].sum()
                print(f"  │  ├─ SoupX {int(frac*100)}%: {n_soup} cells ({n_soup/adata.n_obs*100:.1f}%)")
            
            # Step 4: Simple doublet detection
            print("  ├─ Running simple doublet detection...")
            adata = simple_doublet_detection(adata, quantile_threshold=0.95)
            n_doublets = adata.obs['doubletfinder_flag'].sum()
            print(f"  │  └─ Detected doublets: {n_doublets} ({n_doublets/adata.n_obs*100:.1f}%)")
            
            # Step 5: Combined QC flags for each SoupX threshold
            for frac in [0.05, 0.10, 0.20, 0.25]:
                adata.obs[f'combined_qc_{int(frac*100)}pct_flag'] = (
                    adata.obs['initial_filter_flag'] |
                    adata.obs[f'soupx_{int(frac*100)}pct_flag'] |
                    adata.obs['doubletfinder_flag']
                )
                n_combined = adata.obs[f'combined_qc_{int(frac*100)}pct_flag'].sum()
                print(f"  └─ Combined QC (SoupX {int(frac*100)}%): {n_combined} cells ({n_combined/adata.n_obs*100:.1f}%)")
            
            processed_list.append(adata)
            
        except Exception as e:
            print(f"  ❌ Error processing {sample_id}: {e}")
            print(f"  Skipping this sample...")
            continue
    
    return processed_list

# Run QC pipeline
print("="*60)
print("Starting SAFE QC Pipeline")
print("="*60)
print("\nQC Settings:")
print("  - Initial filter: 5 MAD for counts/genes, 10% MT threshold")
print("  - SoupX thresholds: 5%, 10%, 20%, 25%")
print("  - Doublet detection: Simple quantile-based (top 5%)")
print("="*60)

adata_list_qc = apply_full_qc_pipeline_safe(adata_list)

print("\n" + "="*60)
print(f"QC Pipeline Complete! Processed {len(adata_list_qc)}/{len(adata_list)} samples")
print("="*60)

Starting SAFE QC Pipeline

QC Settings:
  - Initial filter: 5 MAD for counts/genes, 10% MT threshold
  - SoupX thresholds: 5%, 10%, 20%, 25%
  - Doublet detection: Simple quantile-based (top 5%)

[1/34] Processing 1279_3n (8409 cells)...
  ├─ Calculating QC metrics...
  ├─ Applying initial filter (5 MAD, 10% MT)...
  │  └─ Flagged: 1067 cells (12.7%)
  ├─ Running SoupX simulation (5%, 10%, 20%, 25%)...
  │  ├─ SoupX 5%: 659 cells (7.8%)
  │  ├─ SoupX 10%: 971 cells (11.5%)
  │  ├─ SoupX 20%: 1727 cells (20.5%)
  │  ├─ SoupX 25%: 2115 cells (25.2%)
  ├─ Running simple doublet detection...
  │  └─ Detected doublets: 421 (5.0%)
  └─ Combined QC (SoupX 5%): 1694 cells (20.1%)
  └─ Combined QC (SoupX 10%): 2037 cells (24.2%)
  └─ Combined QC (SoupX 20%): 2774 cells (33.0%)
  └─ Combined QC (SoupX 25%): 3136 cells (37.3%)

[2/34] Processing 1296_1n (8520 cells)...
  ├─ Calculating QC metrics...
  ├─ Applying initial filter (5 MAD, 10% MT)...
  │  └─ Flagged: 285 cells (3.3%)
  ├─ Running Sou

---
# 4- Merge all samples

In [5]:
def merge_samples(adata_list, batch_key='sample_id'):
    """
    Merge all samples into one AnnData object
    """
    print("="*60)
    print("Merging Samples")
    print("="*60)
    
    # Check if list is empty
    if len(adata_list) == 0:
        print("❌ No samples to merge!")
        return None
    
    print(f"\nMerging {len(adata_list)} samples...")
    print(f"Total cells before merge: {sum([adata.n_obs for adata in adata_list]):,}")
    
    # Concatenate all samples
    # uns='same' keeps shared unstructured data, batch_key tracks sample origin
    adata_merged = sc.concat(
        adata_list,
        axis=0,
        join='outer',  # Keep all genes (union)
        label=batch_key,
        keys=[adata.obs[batch_key].iloc[0] for adata in adata_list],
        index_unique='_'
    )
    
    print(f"\n✓ Merge complete!")
    print(f"  - Total cells: {adata_merged.n_obs:,}")
    print(f"  - Total genes: {adata_merged.n_vars:,}")
    print(f"  - Samples: {adata_merged.obs[batch_key].nunique()}")
    
    # Summary statistics
    print(f"\n{'='*60}")
    print("Sample Distribution:")
    print(f"{'='*60}")
    sample_counts = adata_merged.obs[batch_key].value_counts().sort_index()
    for sample, count in sample_counts.items():
        sex = adata_merged.obs[adata_merged.obs[batch_key] == sample]['sex'].iloc[0]
        af = adata_merged.obs[adata_merged.obs[batch_key] == sample]['af_status'].iloc[0]
        print(f"  {sample}: {count:>6,} cells (sex={sex}, af={af})")
    
    # AF status distribution
    print(f"\n{'='*60}")
    print("AF Status Distribution:")
    print(f"{'='*60}")
    af_counts = adata_merged.obs['af_status'].value_counts()
    for status, count in af_counts.items():
        print(f"  {status}: {count:>6,} cells ({count/adata_merged.n_obs*100:.1f}%)")
    
    # Sex distribution
    print(f"\n{'='*60}")
    print("Sex Distribution:")
    print(f"{'='*60}")
    sex_counts = adata_merged.obs['sex'].value_counts()
    for sex, count in sex_counts.items():
        print(f"  {sex}: {count:>6,} cells ({count/adata_merged.n_obs*100:.1f}%)")
    
    # QC flags summary
    print(f"\n{'='*60}")
    print("QC Flags Summary:")
    print(f"{'='*60}")
    
    qc_columns = [col for col in adata_merged.obs.columns if 'flag' in col]
    for col in sorted(qc_columns):
        n_flagged = adata_merged.obs[col].sum()
        print(f"  {col}: {n_flagged:>6,} cells ({n_flagged/adata_merged.n_obs*100:.1f}%)")
    
    return adata_merged

# Merge all processed samples
adata_merged = merge_samples(adata_list_qc, batch_key='sample_id')

# Display merged object
print(f"\n{'='*60}")
print("Merged AnnData Object:")
print(f"{'='*60}")
print(adata_merged)

# Check obs columns
print(f"\n{'='*60}")
print("Available metadata columns:")
print(f"{'='*60}")
print(adata_merged.obs.columns.tolist())

Merging Samples

Merging 34 samples...
Total cells before merge: 262,003

✓ Merge complete!
  - Total cells: 262,003
  - Total genes: 38,606
  - Samples: 34

Sample Distribution:
  1279_3n:  8,409 cells (sex=f, af=case)
  1296_1n:  8,520 cells (sex=f, af=case)
  1334_3n:  9,919 cells (sex=f, af=case)
  1357_2n:  4,983 cells (sex=m, af=control)
  1360_1n:  8,451 cells (sex=f, af=case)
  1365_1n:  5,457 cells (sex=m, af=control)
  1369_3n:  3,692 cells (sex=f, af=case)
  1377_3n:  6,034 cells (sex=m, af=case)
  1440_1n:  5,367 cells (sex=f, af=control)
  1465_1n:  8,750 cells (sex=m, af=case)
  1488_2n:  6,981 cells (sex=m, af=control)
  1490_3n: 10,309 cells (sex=f, af=control)
  1498_1n:  4,330 cells (sex=f, af=control)
  1513_1n:  4,281 cells (sex=f, af=control)
  1515_1n:  8,645 cells (sex=f, af=control)
  1540_3n:  9,988 cells (sex=f, af=control)
  1543_1n:  7,945 cells (sex=m, af=case)
  1558_1n:  9,521 cells (sex=f, af=control)
  1561_1n:  2,841 cells (sex=m, af=case)
  1570_1n:  

---
# 5- Save mege adata and format outputs directories

In [7]:
import os
from datetime import datetime

# Create output directory structure
output_base = "/work/archive/farhadie/public_studies/Hill/QC_comparison_analysis"
os.makedirs(output_base, exist_ok=True)

# Create subdirectories
output_dirs = {
    'data': os.path.join(output_base, 'processed_data'),
    'results': os.path.join(output_base, 'results'),
    'figures': os.path.join(output_base, 'figures')
}

for dir_path in output_dirs.values():
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# Save merged data
timestamp = datetime.now().strftime("%Y%m%d")
output_file = os.path.join(output_dirs['data'], f'Hill_merged_with_QC_flags_{timestamp}.h5ad')

print(f"\n{'='*60}")
print("Saving merged data...")
print(f"{'='*60}")
print(f"Output file: {output_file}")

# Save the merged object
adata_merged.write_h5ad(output_file)

# Check file size
file_size_mb = os.path.getsize(output_file) / (1024 * 1024)
print(f"✓ File saved successfully!")
print(f"  File size: {file_size_mb:.1f} MB")

# Also save a metadata summary as CSV for quick reference
metadata_file = os.path.join(output_dirs['data'], f'merged_metadata_summary_{timestamp}.csv')
adata_merged.obs.to_csv(metadata_file)
print(f"\n✓ Metadata saved to: {metadata_file}")

# Create a log file with analysis info
log_file = os.path.join(output_base, f'analysis_log_{timestamp}.txt')
with open(log_file, 'w') as f:
    f.write(f"Hill AF snRNA-seq QC Comparison Analysis\n")
    f.write(f"{'='*60}\n\n")
    f.write(f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Total samples: {adata_merged.obs['sample_id'].nunique()}\n")
    f.write(f"Total cells: {adata_merged.n_obs:,}\n")
    f.write(f"Total genes: {adata_merged.n_vars:,}\n\n")
    
    f.write(f"AF Status Distribution:\n")
    for status, count in adata_merged.obs['af_status'].value_counts().items():
        f.write(f"  {status}: {count:,} cells ({count/adata_merged.n_obs*100:.1f}%)\n")
    
    f.write(f"\nSex Distribution:\n")
    for sex, count in adata_merged.obs['sex'].value_counts().items():
        f.write(f"  {sex}: {count:,} cells ({count/adata_merged.n_obs*100:.1f}%)\n")
    
    f.write(f"\nQC Flags:\n")
    qc_columns = [col for col in adata_merged.obs.columns if 'flag' in col]
    for col in sorted(qc_columns):
        n_flagged = adata_merged.obs[col].sum()
        f.write(f"  {col}: {n_flagged:,} cells ({n_flagged/adata_merged.n_obs*100:.1f}%)\n")

print(f"\n✓ Analysis log saved to: {log_file}")

print(f"\n{'='*60}")
print("Directory Structure:")
print(f"{'='*60}")
print(f"Base: {output_base}")
print(f"  ├─ processed_data/  (merged h5ad files)")
print(f"  ├─ results/         (tables, statistics)")
print(f"  └─ figures/         (plots, visualizations)")

Created directory: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/processed_data
Created directory: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/results
Created directory: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/figures

Saving merged data...
Output file: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/processed_data/Hill_merged_with_QC_flags_20251017.h5ad
✓ File saved successfully!
  File size: 3454.3 MB

✓ Metadata saved to: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/processed_data/merged_metadata_summary_20251017.csv

✓ Analysis log saved to: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis/analysis_log_20251017.txt

Directory Structure:
Base: /work/archive/farhadie/public_studies/Hill/QC_comparison_analysis
  ├─ processed_data/  (merged h5ad files)
  ├─ results/         (tables, statistics)
  └─ figures/         (plots, visualizations)
